<img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" align="right" width="30%">

# A Tour of the Xarray Ecosystem

Xarray is easily extensible.
This means it is easy to add onto to build custom packages that tackle particular computational problems.

These packages can plug in to xarray in various different ways. They may build directly on top of xarray, or they may take advantage of some of xarray's dedicated interfacing features:
- Accessors
- Backend (filetype) entrypoint
- Duck-array wrapping interface
- Metadata attributes
- Flexible indexes (coming soon!)

Here we introduce two popular and widely used extensions that are installable as their own packages (via conda and pip). These packages integrate with xarray using one or more of the features mentioned above.

- [rioxarray](https://corteva.github.io/rioxarray/stable/index.html), for working with geospatial raster data using rasterio
- [pint-xarray](https://pint-xarray.readthedocs.io/en/latest/), for unit-aware computations using pint.

## (Preface) The accessor interface

Before we look at the packages we need to briefly introduce a feature they commonly use: ["xarray accessors"](https://docs.xarray.dev/en/stable/internals/extending-xarray.html).

An accessor is a way of attaching a custom function to xarray types so that it can be called as if it were a method, but while retaining a clear separation between "core" xarray API and custom API.

For example, imagine you're a statistician who regularly uses a special `skewness` function which acts on dataarrays but is only of interest to people in your specific field.

You can create a method which applies this skewness function to an xarray objects, and then register the method under a custom `stats` accessor like this

In [11]:
from scipy.stats import skew

from xarray import register_dataarray_accessor


@register_dataarray_accessor("stats")
class StatsAccessor:
    def __init__(self, da):
        self._da = da
        
    def skewness(self, dim):
        return self._da.reduce(func=skew, dim=dim)

/tmp/ipykernel_24802/2957559907.py:7: AccessorRegistrationWarning: registration of accessor <class '__main__.StatsAccessor'> under name 'stats' for type <class 'xarray.core.dataarray.DataArray'> is overriding a preexisting attribute with the same name.
  class StatsAccessor:


Now we can conveniently access this functionality via the `stats` accessor

In [5]:
import xarray as xr

airtemps = xr.tutorial.load_dataset("air_temperature")

In [12]:
airtemps['air'].stats.skewness(dim="time")

<xarray.DataArray 'air' (lat: 25, lon: 53)>
array([[-0.2931476 , -0.2826622 , -0.27186385, ...,  0.17988019,
         0.19492841,  0.18501355],
       [-0.43506446, -0.41124225, -0.3978455 , ...,  0.20494738,
         0.1514037 ,  0.07443884],
       [-0.30432326, -0.19778351, -0.13770247, ...,  0.11313026,
         0.02851966, -0.08154582],
       ...,
       [-0.29082257, -0.2361239 , -0.18897694, ..., -0.11635019,
        -0.11660235, -0.1263834 ],
       [-0.16569893, -0.15132819, -0.14611286, ..., -0.14533225,
        -0.1657871 , -0.16515452],
       [-0.16930974, -0.18883632, -0.19076061, ..., -0.1121069 ,
        -0.18933195, -0.18690778]], dtype=float32)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0

Notice how the presence of `.stats` clearly differentiates our new "accessor method" from core xarray methods.

This accessor-style syntax is used heavily by the other libraries we are about to cover.

## hvplot via accessors

## Rioxarray via the backend entrypoint
more details about rioxarray here

In [ ]:
import xarray as xr
import rioxarray  # this activates the rio accessor

In [ ]:
# rioxarray example

## cf-xarray via metadata attributes

## Pint via duck array wrapping

more details about pint here


In [ ]:
# to be able to read unit attributes following the CF conventions
import cf_xarray.units  # must be imported before pint_xarray
import pint_xarray

xr.set_options(display_expand_data=False)

## The wider world...

There are many other libraries in the wider xarray ecosystem. For a list of a few packages we particularly like for geoscience work [see here](https://tutorial.xarray.dev/overview/xarray-in-45-min.html#other-cool-packages), and for a [more exhaustive list see here](https://docs.xarray.dev/en/stable/ecosystem.html).